In [1]:
import importlib
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from flearn.utils.model_utils import read_data
import os
import importlib
import random
import numpy as np

from flearn.models import client
from flearn.models import group
from flearn.trainers import grouprox
from flearn.trainers import fedprox
from flearn.trainers import fedavg

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
params = {}
#DATASETS = ['sent140', 'nist', 'shakespeare', 'mnist', 'synthetic_iid', 'synthetic_0_0', 'synthetic_0.5_0.5', 'synthetic_1_1']

"""
Parameters that need to be manually set
"""

#params['dataset'] = 'sent140'
params['dataset'] = 'nist'
params['model'] = 'mlp'
#params['model'] = 'stacked_lstm'
params['optimizer'] = 'grouprox'
params['drop_percent'] = 0

if params['dataset'].startswith('mnist'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.03
  params['mu'] = 1
  params['num_group'] = 3
if params['dataset'] == 'nist':
  params['model_params'] = (26,)
  params['learning_rate'] = 0.003
  params['mu'] = 1
  params['num_group'] = 5
if params['dataset'] == 'sent140':
  params['model_params'] = (25, 2, 100) # seq_len, num_classes, num_hidden
  params['learning_rate'] = 0.3
  params['mu'] = 0.01
  params['num_group'] = 5
if params['dataset'].startswith('synthetic'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.01
  params['mu'] = 1
  params['num_group'] = 5

if params['optimizer']  == 'fedavg':
  pass
if params['optimizer']  == 'fedprox':
  pass
if params['optimizer']  == 'grouprox':
  params['proximal'] = False
  params['RAC'] = True
  params['RCC'] = False
  params['allow_empty'] = True
  
  # TEST Hyperparameters
  params['evenly'] = False
  params['min_clients'] = 2

"""
Common parameters
"""
params['num_rounds'] = 200
params['num_epochs'] = 20
params['batch_size'] = 10
params['eval_every'] = 1
params['clients_per_round'] = 20 ######################################################
params['seed'] = 233
params['agg_lr'] = 0.01


""" Set the output CSV file name """
info = '{}-{}-{}'.format(params['optimizer'], params['dataset'], params['model'])
info2 = ''
suffix = '.csv'
if params['optimizer'] == 'fedavg':
  pass
if params['optimizer'] == 'fedprox':
  pass
if params['optimizer'] == 'grouprox':
  
  info2 = '-NumG{:1d}-Agglr{:.4}-Proximal{:.1}'.format(
    params['num_group'], params['agg_lr'], str(params['proximal']))
  
  if params['RCC'] == True:
    info2 = info2 + '-RCC.{:.1}'.format(str(params['RCC']))
  
  if params['RAC'] == True:
    info2 = info2 + '-RAC.{:.1}-Evenly{:.1}'.format(str(params['RAC']), str(params['evenly']))
  elif params['allow_empty'] == True:
    info2 = info2 + '-Empty{:.1}'.format(str(params['allow_empty']))
  elif params['allow_empty'] == False:
    info2 = info2 + '-Evenly{:.1}'.format(str(params['evenly']))

params['export_filename'] = info + info2 + suffix

print(params['export_filename'])

grouprox-nist-mclr-NumG5-Agglr0.01-ProximalF-RAC.T-EvenlyF.csv


In [3]:
"""
Some preparations
"""

# read data
train_path = os.path.join('data', params['dataset'], 'data', 'train')
test_path = os.path.join('data', params['dataset'], 'data', 'test')
dataset = read_data(train_path, test_path)

# Load model
if params['dataset'].startswith('synthetic'):  # all synthetic datasets use the same model
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'synthetic', params['model'])
elif params['dataset'].startswith('mnist'):
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'mnist', params['model'])
else:
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', params['dataset'], params['model'])
mod = importlib.import_module(model_path)
learner = getattr(mod, 'Model')

# Set seeds
random.seed(1 + params['seed'])
np.random.seed(12 + params['seed'])
tf.set_random_seed(123 + params['seed'])

# Set server
if params['optimizer'] == 'fedavg':
  server = fedavg.Server(params, learner, dataset)  
if params['optimizer'] == 'fedprox':
  server = fedprox.Server(params, learner, dataset)
if params['optimizer'] == 'grouprox':
  server = grouprox.Server(params, learner, dataset)

Using Group prox to Train
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch:   0%|                                                                                      | 0/1 [00:00<?, ?it/s]

200 Clients in Total
20410 Parameters in Total


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Pre-training takes 1.9123432636260986s seconds
SVD takes 0.07723760604858398s seconds
Delta w shape: (100, 20410) Matrix V shape: (20410, 3)
Ternary Cossim Matrix calculation takes 0.008769035339355469s seconds
Clustering takes 0.06634712219238281s seconds
Clustering Results: Counter({1: 23, 0: 23, 3: 21, 2: 17, 4: 16})
Clustering Inertia: 1.006733775138855


In [4]:
server.train()

Epoch:   0%|                                                                                      | 0/1 [00:00<?, ?it/s]

Training with 20 workers ---


Group 0, clients ['f_00154', 'f_00033', 'f_00074']
Group 1, clients ['f_00018', 'f_00107', 'f_00098', 'f_00177', 'f_00007', 'f_00037']
Group 2, clients ['f_00170', 'f_00180', 'f_00080', 'f_00130']
Group 3, clients ['f_00182', 'f_00012', 'f_00152', 'f_00061']
Group 4, clients ['f_00005', 'f_00146', 'f_00125']


Epoch:  35%|██████████████████████████▉                                                  | 7/20 [00:00<00:00, 66.26it/s]

Group 0
At round 0 accuracy: 0.36649214659685864
At round 0 training accuracy: 0.3260061919504644
At round 0 training loss: 2.2774107565082633
Group 1
At round 0 accuracy: 0.3538011695906433
At round 0 training accuracy: 0.3398963730569948
At round 0 training loss: 2.0419706101244595
Group 2
At round 0 accuracy: 0.31232876712328766
At round 0 training accuracy: 0.33545715195922265
At round 0 training loss: 1.9824235395247407
Group 3
At round 0 accuracy: 0.3930635838150289
At round 0 training accuracy: 0.3938584779706275
At round 0 training loss: 2.4083697632731043
Group 4
At round 0 accuracy: 0.33860759493670883
At round 0 training accuracy: 0.36950807360120164
At round 0 training loss: 1.8481984084323078
At round 0 mean test accuracy: 0.35654885654885654 mean train accuracy: 0.3558857560440899
The groups difference are: [1.31396624 1.29598937 1.31914754 1.33039692 1.28824134]
Begin group  0 training


...

Begin group  1 training
Begin group  2 training


Epoch:  25%|███████████████████▎                                                         | 5/20 [00:00<00:00, 49.75it/s]

Begin group  3 training


Epoch:   0%|                                                                                     | 0/20 [00:00<?, ?it/s]

Begin group  4 training


In [5]:
from collections import Counter

group_list = server.group_list
clients = server.clients
g = group_list[-1]
c_list = []
for c in clients:
  if c.group == g:
    c_list.append(c)

cnt = Counter()
for c in c_list:
  cnt += Counter(c.eval_data['y'])
print(cnt)
print(sum(cnt.values()))

Counter({7.0: 107, 6.0: 104, 5.0: 60, 8.0: 45})
316


In [6]:
nd1 = np.arange(10).reshape(5,2)
nd2 = np.arange(10).reshape(5,2)
nd3 = np.array([3.]*10).reshape(5,2)
print((nd2+1.)/2.)
#print(nd2)

print(np.linalg.norm(nd2, axis=1, keepdims=True).shape)

[[0.5 1. ]
 [1.5 2. ]
 [2.5 3. ]
 [3.5 4. ]
 [4.5 5. ]]
(5, 1)


In [7]:
fname = 'sampling_interval.csv'
interval = 5
rounds = 200
with open(fname, 'w') as csv_f:
  csv_f.write(str(1)+'\n')
  for i in range(2, rounds+1):
    if i % interval == 0:
      csv_f.write(str(i))
    elif i == rounds:
      csv_f.write('200')
    csv_f.write('\n')